# Lesson 13: Adaptive Thresholding

## Problem with Global Thresholding

Global thresholding uses a **single threshold T** for the entire image.  
This fails when illumination is **non-uniform** — some regions are bright, others are dark.

## Solution: Adaptive Thresholding

Adaptive thresholding computes a **different threshold for each pixel** based on its local neighborhood.  
This handles uneven lighting effectively.

### OpenCV function:
```python
cv2.adaptiveThreshold(src, maxValue, adaptiveMethod, thresholdType, blockSize, C)
```

| Parameter | Description |
|-----------|-------------|
| `src` | Input grayscale image |
| `maxValue` | Value for pixels that pass the threshold (usually 255) |
| `adaptiveMethod` | How to compute local threshold (`MEAN_C` or `GAUSSIAN_C`) |
| `thresholdType` | `THRESH_BINARY` or `THRESH_BINARY_INV` |
| `blockSize` | Size of neighborhood (must be odd, e.g. 11, 15) |
| `C` | Constant subtracted from the mean (fine-tuning) |

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load a grayscale image
img = cv2.imread('images.jpg', cv2.IMREAD_GRAYSCALE)
if img is None:
    img = cv2.imread('sealion_hero.png', cv2.IMREAD_GRAYSCALE)

# Apply median blur to reduce noise before thresholding
img_blur = cv2.medianBlur(img, 5)

print(f'Image shape: {img.shape}')

## Part 1: Global vs Adaptive — Side by Side

Comparing global threshold and both adaptive methods.

In [ ]:
# Global threshold
_, global_thresh = cv2.threshold(img_blur, 127, 255, cv2.THRESH_BINARY)

# Adaptive Mean: threshold = mean of neighborhood - C
adaptive_mean = cv2.adaptiveThreshold(
    img_blur, 255,
    cv2.ADAPTIVE_THRESH_MEAN_C,
    cv2.THRESH_BINARY,
    blockSize=11, C=2
)

# Adaptive Gaussian: threshold = weighted Gaussian sum of neighborhood - C
adaptive_gaussian = cv2.adaptiveThreshold(
    img_blur, 255,
    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    cv2.THRESH_BINARY,
    blockSize=11, C=2
)

fig, axes = plt.subplots(1, 4, figsize=(16, 4))
axes[0].imshow(img, cmap='gray');              axes[0].set_title('Original'); axes[0].axis('off')
axes[1].imshow(global_thresh, cmap='gray');    axes[1].set_title('Global T=127'); axes[1].axis('off')
axes[2].imshow(adaptive_mean, cmap='gray');    axes[2].set_title('Adaptive Mean'); axes[2].axis('off')
axes[3].imshow(adaptive_gaussian, cmap='gray');axes[3].set_title('Adaptive Gaussian'); axes[3].axis('off')

plt.suptitle('Global vs Adaptive Thresholding', fontsize=12)
plt.tight_layout()
plt.show()

## Part 2: Effect of `blockSize` Parameter

- **Small blockSize** → responds to fine local detail, can be noisy
- **Large blockSize** → smoother result, closer to global thresholding

In [ ]:
block_sizes = [5, 11, 21, 51]

fig, axes = plt.subplots(1, len(block_sizes) + 1, figsize=(18, 4))
axes[0].imshow(img, cmap='gray')
axes[0].set_title('Original')
axes[0].axis('off')

for i, bs in enumerate(block_sizes):
    result = cv2.adaptiveThreshold(
        img_blur, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY,
        blockSize=bs, C=2
    )
    axes[i + 1].imshow(result, cmap='gray')
    axes[i + 1].set_title(f'blockSize={bs}')
    axes[i + 1].axis('off')

plt.suptitle('Adaptive Gaussian — varying blockSize (C=2)', fontsize=12)
plt.tight_layout()
plt.show()

## Part 3: Effect of `C` Constant

The constant `C` is subtracted from the computed local mean.  
- **Higher C** → higher bar for a pixel to be considered foreground → fewer white pixels  
- **Lower C (or negative)** → more pixels pass the threshold → more white pixels

In [ ]:
C_values = [-5, 0, 2, 5, 10]

fig, axes = plt.subplots(1, len(C_values) + 1, figsize=(20, 4))
axes[0].imshow(img, cmap='gray')
axes[0].set_title('Original')
axes[0].axis('off')

for i, c in enumerate(C_values):
    result = cv2.adaptiveThreshold(
        img_blur, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY,
        blockSize=11, C=c
    )
    axes[i + 1].imshow(result, cmap='gray')
    axes[i + 1].set_title(f'C = {c}')
    axes[i + 1].axis('off')

plt.suptitle('Adaptive Gaussian — varying C constant (blockSize=11)', fontsize=12)
plt.tight_layout()
plt.show()

## Part 4: Simulating Non-Uniform Lighting

Let's create a gradient-lit image to demonstrate why global thresholding fails.

In [ ]:
# Simulate non-uniform illumination by adding a gradient overlay
h, w = img.shape
gradient = np.linspace(0, 80, w, dtype=np.float32)
gradient = np.tile(gradient, (h, 1))

uneven = np.clip(img.astype(np.float32) + gradient, 0, 255).astype(np.uint8)

# Global threshold on uneven image — will fail on one side
_, global_uneven = cv2.threshold(uneven, 127, 255, cv2.THRESH_BINARY)

# Adaptive threshold on uneven image — handles it well
adaptive_uneven = cv2.adaptiveThreshold(
    uneven, 255,
    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    cv2.THRESH_BINARY,
    blockSize=21, C=4
)

fig, axes = plt.subplots(1, 4, figsize=(16, 4))
axes[0].imshow(img, cmap='gray');           axes[0].set_title('Original'); axes[0].axis('off')
axes[1].imshow(uneven, cmap='gray');        axes[1].set_title('Non-uniform lighting'); axes[1].axis('off')
axes[2].imshow(global_uneven, cmap='gray'); axes[2].set_title('Global T=127 (fails)'); axes[2].axis('off')
axes[3].imshow(adaptive_uneven, cmap='gray');axes[3].set_title('Adaptive (works!)'); axes[3].axis('off')

plt.suptitle('Why adaptive thresholding is needed for non-uniform lighting', fontsize=11)
plt.tight_layout()
plt.show()

## Summary

| Method | Threshold computed | Best for |
|--------|-------------------|----------|
| Global | Single value for whole image | Uniform, well-lit images |
| Adaptive Mean | Mean of local block | Moderate lighting variation |
| Adaptive Gaussian | Weighted Gaussian of local block | Most real-world images |

### Tips:
- Always **denoise first** with `cv2.medianBlur()` before adaptive thresholding
- `blockSize` controls how **local** the threshold is — start with 11 or 15
- `C = 2` is a good default starting value